In [1]:
# load all the auxiliary functions
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/split.py')
execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/FMAlgoFunctions.py')
execfile('../spark-scripts/FMAlgoMain.py')

In [2]:
import json
import copy

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = '0_1SPLIT_1413851857_sessionsession_1413851857'
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = 's3n://contentwise-research-poli/0_1SPLIT_1413851857_sessionsession_1413851857'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = 'contentwise-research-poli'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 5
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = "split2.split"
conf['general']['bucketName'] = 'contentwise-research-poli'
conf['general']['tmpDir'] = "/tmp"

conf['algo'] = {}
conf['algo']['name'] = 'LibFM_fixed'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']['collaborativity'] = 'knn' #individual, (all), knn
conf['algo']['props']['neighborhoodSize'] = 2 #neighborhood size for the knn collaborativity (NOTE: it includes the user itself! set to 2 for the closest neighbor)
conf['algo']['props']['duplicatePolicy'] = 'unique_max' #none, unique_max, unique_min
conf['algo']['props']['historyAggrPolicy'] = 'count' #count, avg, max, min, [exp_decay and others to be implemented]
conf['algo']['props']['trainGenPolicy'] = 'leaveoneout' #leaveoneout, sequential
conf['algo']['props']['latentSize'] = 2
conf['algo']['props']['libFMbin'] = "/home/massimo/recsys/libfm/bin/./libFM"
conf['algo']['props']['libFMopt'] = "-task c -dim '1,1,25' -iter 50" 
conf['algo']['props']['shuffleTraining'] = True
conf['algo']['props']['maxRequestsPerUser'] = 10
# ****** END EXAMPLE ****************#

In [ ]:
historyAggrPolicies = ['count', 'avg']
trainGenPolicies = ['leaveoneout', 'sequential']
#collaborativityTypes = ['individual', 'knn', 'all']
collaborativityTypes = ['individual', 'knn']
neighSizes = [2**i for i in xrange(1,7)]
latentSizes = [2**i for i in xrange(2,7)]

coll_neigh = [(c, 1) for c in collaborativityTypes if c is not 'knn']
if 'knn' in collaborativityTypes:
    coll_neigh += [('knn', s) for s in neighSizes]

#for exclude in [True,False]:
#for exclude in [True]:
#    conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude%s' % exclude
#    splitter(conf)
train, test = loadDataset(conf)

for historyAggrPolicy in historyAggrPolicies:
    conf['algo']['props']['historyAggrPolicy'] = historyAggrPolicy
    trainSessionHistoryAggr, testRequestsHistoryAggr = sessionAggregation(train, test, conf)
    
    for trainGenPolicy in trainGenPolicies:
        print 
        conf['algo']['props']['trainGenPolicy'] = trainGenPolicy
        training = generateTraining(trainSessionHistoryAggr, conf)
        
        for coll, neigh_size in coll_neigh:
            conf['algo']['props']['collaborativity'] = coll
            conf['algo']['props']['neighborhoodSize'] = neigh_size
            trainSessionsPerUser, testRequestsPerUser = filterTrainSessionsPerUser(training, trainSessionHistoryAggr, testRequestsHistoryAggr, conf)
            
            for latentSize in latentSizes:
                conf['algo']['props']['latentSize'] = latentSize
                conf['algo']['props']['libFMopt'] = "-task c -dim '1,1,%d' -iter 50" % latentSize
                recomm = generateRecommendationsLibFM(trainSessionsPerUser, testRequestsPerUser, test, conf)
                try:
                    saveRecommendations(conf, recomm, overwrite=True)
                    computeMetrics(conf)
                except Exception as e:
                    print e
                    pass

In [3]:
# DEBUG SCRIPT

historyAggrPolicies = ['avg']
trainGenPolicies = ['leaveoneout']
#collaborativityTypes = ['individual', 'knn', 'all']
collaborativityTypes = ['individual']
neighSizes = [4]
latentSizes = [4]

coll_neigh = [(c, 1) for c in collaborativityTypes if c is not 'knn']
if 'knn' in collaborativityTypes:
    coll_neigh += [('knn', s) for s in neighSizes]


#for exclude in [True]:
#    conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude' + str(exclude)
#    splitter(conf)
train, test = loadDataset(conf)

for historyAggrPolicy in historyAggrPolicies:
    conf['algo']['props']['historyAggrPolicy'] = historyAggrPolicy
    trainSessionHistoryAggr, testRequestsHistoryAggr = sessionAggregation(train, test, conf)
    
    for trainGenPolicy in trainGenPolicies:
        conf['algo']['props']['trainGenPolicy'] = trainGenPolicy
        training = generateTraining(trainSessionHistoryAggr, conf)
        
        for coll, neigh_size in coll_neigh:
            conf['algo']['props']['collaborativity'] = coll
            conf['algo']['props']['neighborhoodSize'] = neigh_size
            trainSessionsPerUser, testRequestsPerUser = filterTrainSessionsPerUser(training, trainSessionHistoryAggr, testRequestsHistoryAggr, conf)
            
            for latentSize in latentSizes:
                conf['algo']['props']['latentSize'] = latentSize
                conf['algo']['props']['libFMopt'] = "-task c -method als -dim '1,1,%d' -iter 50" % latentSize
                recomm = generateRecommendationsLibFM(trainSessionsPerUser, testRequestsPerUser, test, conf)
                saveRecommendations(conf, recomm, overwrite=True)
                computeMetrics(conf)


KeyboardInterrupt: 

split2.split/0_1SPLIT_1413851857_sessionsession_1413851857/Rec/LibFM_fixed_leaveoneout#unique_max#4#avg#1#individual successfully deleted.
